In [10]:
import pandas as pd
import datetime as datetime
from datetime import timedelta
from google.cloud import bigquery
import os

def load_sales_initial():
    
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/zachquinn/Downloads/ornate-reef-332816-a7425b762ba2.json"

    sales_1_5 = pd.read_csv('sales_2022_01_05.csv')
    sales_1_5_df = pd.DataFrame(sales_1_5)
    sales_1_5_df['date'] = pd.to_datetime(sales_1_5_df['date']).dt.floor('d')
    sales_1_5_df['last_updated_on'] = pd.Timestamp.now()
    sales_1_5_df

    client = bigquery.Client()
    dataset_id = 'sales'
    table_id = 'totals'

    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)

    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = 'WRITE_TRUNCATE'
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.autodetect = True
    job_config.ignore_unknown_values = True

    job = client.load_table_from_dataframe(
        sales_1_5_df,
        table_ref,
        location='US',
        job_config=job_config
    )

    
load_sales_initial()

In [14]:
def load_sales_append():
    
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/zachquinn/Downloads/ornate-reef-332816-a7425b762ba2.json"

    sales_1_6 = pd.read_csv('sales_2022_01_06.csv')
    sales_1_6_df = pd.DataFrame(sales_1_6)
    sales_1_6_df['date'] = pd.to_datetime(sales_1_6_df['date']).dt.floor('d')
    sales_1_6_df['last_updated_on'] = pd.Timestamp.now()

    client = bigquery.Client()
    dataset_id = 'sales'
    table_id = 'totals'

    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)

    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = 'WRITE_APPEND'
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.autodetect = True
    job_config.ignore_unknown_values = True

    job = client.load_table_from_dataframe(
        sales_1_6_df,
        table_ref,
        location='US',
        job_config=job_config
    )
    
    query_string = """
    
    SELECT
  * EXCEPT(max_timestamp)
FROM (
  SELECT
    date,
    user,
    sales,
    last_updated_on,
    MAX(last_updated_on) OVER(PARTITION BY date) AS max_timestamp
  FROM
    `ornate-reef-332816.sales.totals` )
WHERE
  last_updated_on = max_timestamp
ORDER BY
  date DESC
    
    """
    client = bigquery.Client()
    dataset_id = 'sales'
    table_id = 'totals'

    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)

    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = 'WRITE_TRUNCATE'
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.autodetect = True
    job_config.ignore_unknown_values = True

    job = client.load_table_from_dataframe(
        sales_1_6_df,
        table_ref,
        location='US',
        job_config=job_config
    )
    job.result()
    
    return 'Finished'

load_sales_append()

'Finished'